In [138]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import (QSystemTrayIcon, QApplication, QLabel, QMainWindow, QPushButton, QWidget, 
                             QVBoxLayout, QTextEdit, QDialog, QComboBox, QAction, QLineEdit, QLabel, QInputDialog)
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt, QCoreApplication, QSettings, QPoint
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import inspect
import ctypes
from io import BytesIO
import win32clipboard
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
import qt_utils

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [33]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    try:
        return texts[0].description
    except IndexError:
        print('no text in image')

In [39]:
class ocrWidget(QtWidgets.QWidget):
#     clicked = QtCore.pyqtSignal(object)
    
    def __init__(self, parent = None, lang_hint = []):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.parent = parent
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )
        self.lang_hint = lang_hint
        print(self.lang_hint)

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
#         self.close()
        self.closeAndReturn()
        

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
    
    def closeAndReturn(self):
        self.close()
        self.parent.setWindowOpacity(1.)
        return

    def mouseReleaseEvent(self, event):
        previous = self.parent
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        if (y1 == y2 and x1 == x2):
            print("no region selected")
            self.closeAndReturn()
        
        with mss.mss() as sct:
            # The screen part to capture            
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)
            sct_img = sct.grab(monitor)
            try:
                img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")
                img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
                im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
#                 ocr = dtlh(thresh, ["hi"])
                ocr = dtlh(thresh, self.lang_hint)
                if ocr:
#                     send_text_clipboard(ocr)
                    print(ocr)
                self.parent.setWindowOpacity(1.)
            except ValueError:
                print("invalid region selection. try again")
                self.closeAndReturn()

In [173]:

lang_hints_dict = {u'Afrikaans - Afrikaans': "af", 
u'Albanian - shqip': "sq", 
u'Arabic - العربية': "ar", 
u'Armenian - Հայ': "hy", 
u'Belorussian - беларускі': "be", 
u'Bulgarian - български': "bg", 
u'Catalan - Català': "ca", 
u'Chinese - 普通话': "zh", 
u'Croatian - Hrvatski': "hr", 
u'Czech - Čeština': "cs", 
u'Danish - Dansk': "da", 
u'Dutch - Nederlands': "nl", 
u'English  - English': "en", 
u'Estonian - Eesti keel': "et", 
u'Filipino - Filipino': "fil (or tl)", 
u'Finnish - Suomi': "fi", 
u'French  - Français': "fr", 
u'German - Deutsch': "de", 
u'Greek - Ελληνικά': "el", 
u'Gujarati - ગુજરાતી': "gu", 
u'Hebrew - עברית': "iw", 
u'Hindi - हिन्दी': "hi", 
u'Hungarian - Magyar': "hu", 
u'Icelandic - Íslenska': "is", 
u'Indonesian - Bahasa Indonesia': "id", 
u'Italian - Italiano': "it", 
u'Japanese - 日本語': "ja", 
u'Kannada - ಕನ್ನಡ': "kn", 
u'Khmer - ភាសាខ្មែរ': "km", 
u'Korean - 한국어': "ko", 
u'Lao - ລາວ': "lo", 
u'Latvian - Latviešu': "lv", 
u'Lithuanian - Lietuvių': "lt", 
u'Macedonian - Македонски': "mk", 
u'Malay - Bahasa Melayu': "ms", 
u'Malayalam - മലയാളം': "ml", 
u'Marathi - मराठी': "mr", 
u'Nepali - नेपाली': "ne", 
u'Norwegian - Norsk': "no", 
u'Persian - فارسی': "fa", 
u'Polish - Polski': "pl", 
u'Portuguese - Português': "pt", 
u'Punjabi - ਪੰਜਾਬੀ': "pa", 
u'Romanian - Română': "ro", 
u'Russian - Русский': "ru", 
u'Russian - Русский (старая орфография)': "ru-PETR1708", 
u'Serbian - Српски': "sr", 
u'Serbian - Српски (латиница)': "sr-Latn", 
u'Slovak - Slovenčina': "sk", 
u'Slovenian - Slovenščina': "sl", 
u'Spanish - Español': "es", 
u'Swedish - Svenska': "sv", 
u'Tamil - தமிழ்': "ta", 
u'Telugu - తెలుగు': "te", 
u'Thai - ไทย': "th", 
u'Turkish - Türkçe': "tr", 
u'Ukrainian - Українська': "uk", 
u'Vietnamese - Tiếng Việt': "vi", 
u'Yiddish - Yiddish': "yi"}

class Central(qt_utils.QMainWindow):
    def __init__(self, parent=None):
        super(Central, self).__init__(parent)
        self.pushButton = QPushButton("OCR")
#         self.settingsButton = QPushButton("settings")
        self.pushButton.clicked.connect(self.on_pushButton_clicked)
#         self.settingsButton.clicked.connect(self.on_settingsButton_clicked)
        self.dialogs = list()
#         self.settings_menu = list()
        self.combo = QComboBox(self)
        self.populate_dict(self.combo)
        self.codes = set(lang_hints_dict.values())
        self.combo.activated.connect(self.handleActivated)
        self.lang_hint = []    
        self.multiButton = QPushButton("OCR Multiple languages")
        self.multiButton.clicked.connect(self.sd)
        layout = QVBoxLayout()
        layout.addWidget(self.combo)
        layout.addWidget(self.pushButton)
#         layout.addWidget(self.line)
        layout.addWidget(self.multiButton)
#         layout.addWidget(self.settingsButton)
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
#         self.getText()

        self._gui_restore()
#         self.populate_dict(self.combo)
    
    def on_pushButton_clicked(self):
        dialog = ocrWidget(self, self.lang_hint)
        self.dialogs.append(dialog)
        dialog.show()
        self.setWindowOpacity(0.)
    
    def on_settingsButton_clicked(self):
        settings_menu = Settings(self)
        self.settings_menu.append(settings_menu)
        settings_menu.show()
        
    def on_multiButton_clicked(self):
#         print(self.line.text())
        self.getText()
        
    def handleActivated(self, index):
        self.lang_hint = [self.combo.itemData(index)]
        print(self.combo.itemText(index))
        print(self.lang_hint)
        
    def populate_dict(self, combo : QComboBox):
        for key, value in lang_hints_dict.items():
            self.combo.addItem(key, value)
            
    def sd(self):
        info_string = """To enter multiple languages, enter the language code in a comma seperate list in order of importance. For example, 
    if you want to recognize a document with Arabic, English, and French in it, and (Arabic being most and French least important)
    enter ar,en,fr. For the full list of language codes see https://cloud.google.com/vision/docs/languages#supported-langs
    """
        text , ok = QInputDialog.getText(self,'InputDialog', info_string)
        if ok:
            lh = text.replace(" ", "").split(',')
            self.lang_hint.clear()
            for elem in lh:
                if elem not in self.codes:
                    print("the language code {0} cannot be found! check your codes!".format(elem))
                    return
                else:
                    self.lang_hint.append(elem)
#             print(self.lang_hint)
            self.on_pushButton_clicked()

    
def main():
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    main = Central()
    main.show()
    sys.exit(app.exec_())
    
if __name__ == '__main__':
    main()

['ar', 'en', 'fr']
الراكبة جورجيت بشیر (۹
عاما) وهي کندية من اص
مصري
سري، داخل صالة يرانزی
مطار القاهرة بعد أول زيارة
الى مصر متنثرة بإصابتها بهبو
حاد في الدورة الدموية ق
دقائق من صعودها إلى الطائ
ال
1,7



SystemExit: 0

In [156]:
class Settings(qt_utils.QMainWindow):
    def __init__(self, parent=None):
        super(Settings, self).__init__(parent)
#         self.settings = parent.settings
        self.combo1 = QComboBox(self)
        self.combo2 = QComboBox(self)
        self.combo3 = QComboBox(self)
        self.populate_dict(self.combo1)
        self.populate_dict(self.combo2)
        self.populate_dict(self.combo3)
        self.combo1.activated.connect(self.handleActivated1)
        self.combo2.activated.connect(self.handleActivated2)
        self.combo3.activated.connect(self.handleActivated3)
        layout = QVBoxLayout()
        layout.addWidget(self.combo1)
        layout.addWidget(self.combo2)
        layout.addWidget(self.combo3)
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        self._gui_restore()
    
    def handleActivated1(self, index):
        self.lh1 = [self.combo1.itemData(index)]
    def handleActivated2(self, index):
        self.lh2 = [self.combo2.itemData(index)]
    def handleActivated3(self, index):
        self.lh3 = [self.combo3.itemData(index)]
    
    def populate_dict(self, combo : QComboBox):
        for key, value in lang_hints_dict.items():
            combo.addItem(key, value)

    
#     def connectCombos(self, num):
#         if num == 1:
#             self.combo1.activated.connect(self.handleActivated)

In [ ]:

#somehow remeber the favorite langs selected
#create keyboard shorcuts??
#consider logic for setting multuple lang hints (i.e. checkbox) 
#settings menu to set quick access langs i.e. english, spanish, french 